In [1]:
import os

In [2]:
%pwd

'c:\\Users\\deepa\\Panda 2024\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\deepa\\Panda 2024\\Text-Summarizer-Project'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
from textSummarizer.constant import *
from textSummarizer.utils.common import create_directories,read_yaml

In [24]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config



        

In [26]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [27]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with the following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-19 00:12:57,930: INFO: common: yaml files: config\config.yaml loaded successfully]
[2024-09-19 00:12:57,942: INFO: common: yaml files: params.yaml loaded successfully]
[2024-09-19 00:12:57,944: INFO: common: created directory at: artifacts]
[2024-09-19 00:12:57,947: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-19 00:12:59,751: INFO: 1518879251: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DDC1:8B9F:4C2C92:5F02AC:66EB1F32
Accept-Ranges: bytes
Date: Wed, 18 Sep 2024 18:42:59 GMT
Via: 1.1 varnish
X-Served-By: cache-del21742-DEL
X-Cache